In [1]:
import os
import torch

# Check for the availability of CUDA-enabled GPU
if torch.cuda.is_available():
    # Set device to GPU
    torch.cuda.set_device(0)
    device = torch.device("cuda")
    print("Switched to GPU")
else:
    # Use CPU if no GPU is available
    device = torch.device("cpu")
    print("No GPU available, using CPU")

print("Using device:", device)


Switched to GPU
Using device: cuda


In [2]:
import os

judgement_folder = os.path.join(os.getcwd(), "judgement")
summary_folder = os.path.join(os.getcwd(), "summary")

judgements = []
summaries = []

for filename in os.listdir(judgement_folder):
    with open(os.path.join(judgement_folder, filename), 'r', encoding='utf-8') as file:
        judgements.append(file.read())
    
    with open(os.path.join(summary_folder, filename), 'r', encoding='utf-8') as file:
        summaries.append(file.read())


In [3]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)


c:\Users\dell\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
inputs = tokenizer(judgements, return_tensors="pt", truncation=True, max_length=1024, padding=True)
labels = tokenizer(summaries, return_tensors="pt", truncation=True, max_length=150, padding=True)


In [11]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler

from transformers import AdamW, get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

# ... (previous code)

# Model and optimizer initialization
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(inputs["input_ids"]))

# Hyperparameters
batch_size = 2  # Adjust this value
gradient_accumulation_steps = 4  # Adjust this value
model = model.to("cuda")
inputs = {key: value.to("cuda") for key, value in inputs.items()}
labels = {key: value.to("cuda") for key, value in labels.items()}

# Mixed precision training
scaler = GradScaler()

# Training loop with tqdm
num_batches = len(inputs["input_ids"]) // batch_size
progress_bar = tqdm(total=num_epochs * num_batches, desc="Training")

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    for i in range(0, len(inputs["input_ids"]), batch_size):
        input_batch = {key: value[i:i+batch_size] for key, value in inputs.items()}
        label_batch = {key: value[i:i+batch_size] for key, value in labels.items()}

        # Move input_batch to GPU
        input_batch = {key: value.to("cuda") for key, value in input_batch.items()}

        with autocast():
            outputs = model(**input_batch, labels=label_batch["input_ids"])
            loss = outputs.loss / gradient_accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        progress_bar.update(1)
        progress_bar.set_postfix({"Epoch": epoch + 1, "Loss": loss.item()})

    scheduler.step()

progress_bar.close()

# Save the model
model.save_pretrained("legal_ease_model")

Training:   0%|          | 0/3515 [01:16<?, ?it/s]
c:\Users\dell\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\bart\modeling_bart.py:590: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\dell\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `

In [12]:
model.save_pretrained("legal_ease_model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [16]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the saved model
model = BartForConditionalGeneration.from_pretrained("legal_ease_model")
tokenizer = BartTokenizer.from_pretrained("legal_ease_model")

# Read the contents of the text file
file_path = "1.txt"
with open(file_path, "r", encoding="utf-8") as file:
    legal_document = file.read()

# Tokenize and generate a summary
input_ids = tokenizer.encode(legal_document, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(input_ids, max_length=150, length_penalty=2.0, num_beams=4, early_stopping=True)

generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Generated Summary:", generated_summary)


OSError: Can't load tokenizer for 'legal_ease_model'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'legal_ease_model' is the correct path to a directory containing all relevant files for a BartTokenizer tokenizer.